In [1]:
import pandas as pd, numpy as np
import os, re

In [2]:
tweets_data = pd.read_csv('TwitterHate.csv')
tweets_data.head(20)

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation
5,6,0,[2/2] huge fan fare and big talking before the...
6,7,0,@user camping tomorrow @user @user @user @use...
7,8,0,the next school year is the year for exams.ð...
8,9,0,we won!!! love the land!!! #allin #cavs #champ...
9,10,0,@user @user welcome here ! i'm it's so #gr...


In [3]:
tweets_data.label.value_counts(normalize=True)

0    0.929854
1    0.070146
Name: label, dtype: float64

In [4]:
tweets_data.tweet.sample().values[0]

" @user thank you @user for my warm welcome this morning. i enjoyed hearing about sip's for next year. can't wait!  "

In [5]:
tweets = tweets_data.tweet.values

In [6]:
len(tweets)

31962

In [7]:
tweets[:5]

array([' @user when a father is dysfunctional and is so selfish he drags his kids into his dysfunction.   #run',
       "@user @user thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx.    #disapointed #getthanked",
       '  bihday your majesty',
       '#model   i love u take with u all the time in urð\x9f\x93±!!! ð\x9f\x98\x99ð\x9f\x98\x8eð\x9f\x91\x84ð\x9f\x91\x85ð\x9f\x92¦ð\x9f\x92¦ð\x9f\x92¦  ',
       ' factsguide: society now    #motivation'], dtype=object)

In [8]:
tweets_lower = [twt.lower() for twt in tweets]

In [9]:
tweets_lower[:5]

[' @user when a father is dysfunctional and is so selfish he drags his kids into his dysfunction.   #run',
 "@user @user thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx.    #disapointed #getthanked",
 '  bihday your majesty',
 '#model   i love u take with u all the time in urð\x9f\x93±!!! ð\x9f\x98\x99ð\x9f\x98\x8eð\x9f\x91\x84ð\x9f\x91\x85ð\x9f\x92¦ð\x9f\x92¦ð\x9f\x92¦  ',
 ' factsguide: society now    #motivation']

In [10]:
import re

In [11]:
tweets_nouser = [re.sub("@\w+","", twt) for twt in tweets_lower]

In [12]:
tweets_nouser[:5]

['  when a father is dysfunctional and is so selfish he drags his kids into his dysfunction.   #run',
 "  thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx.    #disapointed #getthanked",
 '  bihday your majesty',
 '#model   i love u take with u all the time in urð\x9f\x93±!!! ð\x9f\x98\x99ð\x9f\x98\x8eð\x9f\x91\x84ð\x9f\x91\x85ð\x9f\x92¦ð\x9f\x92¦ð\x9f\x92¦  ',
 ' factsguide: society now    #motivation']

In [13]:
tweets_nourl = [re.sub("\w+://\S+","", twt) for twt in tweets_nouser]

In [14]:
tweets_nourl[:5]

['  when a father is dysfunctional and is so selfish he drags his kids into his dysfunction.   #run',
 "  thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx.    #disapointed #getthanked",
 '  bihday your majesty',
 '#model   i love u take with u all the time in urð\x9f\x93±!!! ð\x9f\x98\x99ð\x9f\x98\x8eð\x9f\x91\x84ð\x9f\x91\x85ð\x9f\x92¦ð\x9f\x92¦ð\x9f\x92¦  ',
 ' factsguide: society now    #motivation']

In [15]:
from nltk.tokenize import TweetTokenizer

In [16]:
tkn = TweetTokenizer()

In [17]:
print(tkn.tokenize(tweets_nourl[0]))

['when', 'a', 'father', 'is', 'dysfunctional', 'and', 'is', 'so', 'selfish', 'he', 'drags', 'his', 'kids', 'into', 'his', 'dysfunction', '.', '#run']


In [18]:
tweet_token = [tkn.tokenize(sent) for sent in tweets_nourl]
print(tweet_token[0])

['when', 'a', 'father', 'is', 'dysfunctional', 'and', 'is', 'so', 'selfish', 'he', 'drags', 'his', 'kids', 'into', 'his', 'dysfunction', '.', '#run']


In [19]:
from nltk.corpus import stopwords
from string import punctuation

In [20]:
stop_nltk = stopwords.words("english")
stop_punct = list(punctuation)

In [21]:
stop_punct.extend(['...','``',"''",".."])


In [22]:
stop_context = ['rt', 'amp']

In [23]:
stop_final = stop_nltk + stop_punct + stop_context

In [24]:
def del_stop(sent):
    return [re.sub("#","",term) for term in sent if ((term not in stop_final) & (len(term)>1))]


In [25]:
del_stop(tweet_token[4])

['factsguide', 'society', 'motivation']

In [26]:
tweets_clean = [del_stop(tweet) for tweet in tweet_token]

In [27]:
from collections import Counter

In [28]:
term_list = []
for tweet in tweets_clean:
    term_list.extend(tweet)

In [29]:
res = Counter(term_list)
res.most_common(20)

[('love', 2748),
 ('day', 2276),
 ('happy', 1684),
 ('time', 1131),
 ('life', 1118),
 ('like', 1047),
 ("i'm", 1018),
 ('today', 1013),
 ('new', 994),
 ('thankful', 946),
 ('positive', 931),
 ('get', 917),
 ('good', 862),
 ('people', 859),
 ('bihday', 844),
 ('one', 783),
 ('see', 762),
 ("can't", 747),
 ('smile', 712),
 ('go', 658)]

In [30]:
tweets_clean[0]

['father', 'dysfunctional', 'selfish', 'drags', 'kids', 'dysfunction', 'run']

In [31]:
tweets_clean = [" ".join(tweet) for tweet in tweets_clean]

In [32]:
tweets_clean[0]

'father dysfunctional selfish drags kids dysfunction run'

In [33]:
len(tweets_clean)

31962

In [34]:
len(tweets_data.label)

31962

In [35]:
X = tweets_clean
y = tweets_data.label.values

In [36]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state=42)

In [37]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [38]:
vectorizer = TfidfVectorizer(max_features = 31962)

In [39]:
len(X_train), len(X_test)

(22373, 9589)

In [40]:
X_train_bow = vectorizer.fit_transform(X_train)

X_test_bow = vectorizer.transform(X_test)

In [41]:
X_train_bow.shape, X_test_bow.shape

((22373, 31962), (9589, 31962))

In [42]:
from sklearn.linear_model import LogisticRegression

In [43]:
logreg = LogisticRegression()


In [44]:
logreg.fit(X_train_bow, y_train)

LogisticRegression()

In [45]:
y_train_pred = logreg.predict(X_train_bow)
y_test_pred = logreg.predict(X_test_bow)

In [46]:
from sklearn.metrics import accuracy_score, classification_report

In [47]:
accuracy_score(y_train, y_train_pred)

0.9516381352523131

In [48]:
print(classification_report(y_train, y_train_pred))

              precision    recall  f1-score   support

           0       0.95      1.00      0.97     20815
           1       0.97      0.31      0.47      1558

    accuracy                           0.95     22373
   macro avg       0.96      0.66      0.72     22373
weighted avg       0.95      0.95      0.94     22373



In [49]:
logreg = LogisticRegression(class_weight="balanced")

In [50]:
logreg.fit(X_train_bow, y_train)

LogisticRegression(class_weight='balanced')

In [51]:
y_train_pred = logreg.predict(X_train_bow)
y_test_pred = logreg.predict(X_test_bow)

In [52]:
accuracy_score(y_train, y_train_pred)

0.9753274035667993

In [53]:
print(classification_report(y_train, y_train_pred))

              precision    recall  f1-score   support

           0       1.00      0.97      0.99     20815
           1       0.74      1.00      0.85      1558

    accuracy                           0.98     22373
   macro avg       0.87      0.98      0.92     22373
weighted avg       0.98      0.98      0.98     22373



In [54]:
from sklearn.model_selection import GridSearchCV, StratifiedKFold

In [55]:
# Create the parameter grid based on the results of random search 
param_grid = {
    'C': [0.01,0.1,1,10,100],
    'penalty': ["l1","l2"]
}

In [56]:
classifier_lr = LogisticRegression(class_weight="balanced")

In [57]:
grid_search = GridSearchCV(estimator = classifier_lr, param_grid = param_grid, 
                          cv = StratifiedKFold(4), n_jobs = -1, verbose = 1, scoring = "recall" )

In [58]:
grid_search.fit(X_train_bow, y_train)

Fitting 4 folds for each of 10 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:    6.4s finished


GridSearchCV(cv=StratifiedKFold(n_splits=4, random_state=None, shuffle=False),
             estimator=LogisticRegression(class_weight='balanced'), n_jobs=-1,
             param_grid={'C': [0.01, 0.1, 1, 10, 100], 'penalty': ['l1', 'l2']},
             scoring='recall', verbose=1)

In [59]:
grid_search.best_estimator_

LogisticRegression(C=1, class_weight='balanced')

In [60]:
y_test_pred = grid_search.best_estimator_.predict(X_test_bow)

In [61]:
y_train_pred = grid_search.best_estimator_.predict(X_train_bow)

In [62]:
print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

           0       0.98      0.96      0.97      8905
           1       0.57      0.75      0.65       684

    accuracy                           0.94      9589
   macro avg       0.78      0.85      0.81      9589
weighted avg       0.95      0.94      0.95      9589



# Naive bayes

In [63]:
from sklearn.naive_bayes import MultinomialNB


In [64]:
Multinom = MultinomialNB()


In [65]:
Multinom.fit(X_train_bow, y_train)

MultinomialNB()

In [66]:
y_train_pred = Multinom.predict(X_train_bow)
y_test_pred = Multinom.predict(X_test_bow)

In [67]:
accuracy_score(y_train, y_train_pred)

0.9452911992133375

In [68]:
print(classification_report(y_train, y_train_pred))

              precision    recall  f1-score   support

           0       0.94      1.00      0.97     20815
           1       1.00      0.21      0.35      1558

    accuracy                           0.95     22373
   macro avg       0.97      0.61      0.66     22373
weighted avg       0.95      0.95      0.93     22373



# Random forest

In [69]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()

In [70]:
rf.fit(X_train_bow, y_train)

RandomForestClassifier()

In [71]:
y_train_pred = rf.predict(X_train_bow)
y_test_pred = rf.predict(X_test_bow)

In [72]:
accuracy_score(y_train, y_train_pred)

0.9995083359406427

In [73]:
print(classification_report(y_train, y_train_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     20815
           1       1.00      0.99      1.00      1558

    accuracy                           1.00     22373
   macro avg       1.00      1.00      1.00     22373
weighted avg       1.00      1.00      1.00     22373



# KNN ALGORITM

In [74]:
from sklearn.neighbors import NearestCentroid

In [75]:
knn = NearestCentroid()

In [76]:
knn.fit(X_train_bow, y_train)

NearestCentroid()

In [77]:
y_train_pred = knn.predict(X_train_bow)
y_test_pred = knn.predict(X_test_bow)

In [78]:
accuracy_score(y_train, y_train_pred)

0.9430116658472266

In [79]:
print(classification_report(y_train, y_train_pred))

              precision    recall  f1-score   support

           0       0.98      0.96      0.97     20815
           1       0.57      0.74      0.64      1558

    accuracy                           0.94     22373
   macro avg       0.77      0.85      0.81     22373
weighted avg       0.95      0.94      0.95     22373

